In [1]:
# NB: This is for spark running on parquet files converted from
#     baconbits skims. This is a prototype, there is lots of boilerplate.
#     We're making it better :-)

import pyspark.sql
import os
import sys
from pyarrow.compat import guid
from coffea.processor.spark.detail import _spark_initialize

# The following line is necessary because we're working in a
# virtualenv. Without it, executors will use the wrong interpreter!
#os.environ['PYSPARK_PYTHON'] = sys.executable

#    .config("spark.driver.extraClassPath","/home/cms.lgray/sparkMeasure/target/scala-2.12/spark-measure_2.12-0.14-SNAPSHOT.jar") \
#    .master('local[1]') \
#    .master('spark://cmsspark-submit.fnal.gov:7077') \
conf = pyspark.sql.SparkSession.builder \
    .appName('decaf-%s' % guid()) \
    .master('spark://cmsspark-submit.fnal.gov:7077') \
    .config('spark.driver.cores',8) \
    .config('spark.driver.memory','10g') \
    .config('spark.executor.memory', "20g") \
    .config('spark.debug.maxToStringFields',1000) \
    .config('spark.executorEnv.XRD_RUNFORKHANDLER','1') \
    .config('spark.executorEnv.X509_PROXY',"%s" % os.environ['X509_USER_PROXY']) \
    .config('spark.files','%s' % os.environ['X509_USER_PROXY']) \
    .config('spark.sql.execution.arrow.enabled','true') \
    .config("spark.blockManager.port", "8000") \
    .config('spark.shuffle.service.enabled','true') \
    .config('spark.scheduler.mode', 'FAIR') \
    .config('spark.executor.instances','1') \
    .config('spark.dynamicAllocation.enabled','true') \
    .config('spark.dynamicAllocation.executorIdleTimeout','1m') \
    .config('spark.dynamicAllocation.cachedExecutorIdleTimeout','10m') \
    .config('spark.dynamicAllocation.minExecutors','1') \
    .config('spark.dynamicAllocation.maxExecutors','80') \
    .config('spark.sql.execution.arrow.pyspark.enabled','true') \
    .config('spark.sql.execution.arrow.maxRecordsPerBatch', 200000) \
    .config('spark.hadoop.fs.xrootd.read.buffer', 100 * 1048576) \
    .config('spark.hadoop.fs.xrootd.write.buffer', 100 * 1048576) \
    .config('spark.driver.extraClassPath','file:///opt/hadoop-xrootd/hadoop-xrootd-1.0.4-jar-with-dependencies.jar') \
    .config('spark.executor.extraClassPath','file:///opt/hadoop-xrootd/hadoop-xrootd-1.0.4-jar-with-dependencies.jar')
session = _spark_initialize(config=conf,log_level='WARN',laurelin_version='0.4.2-SNAPSHOT')
sc = session.sparkContext
sc.setLogLevel("WARN")
spark = session

#from sparkmeasure import TaskMetrics
#taskmetrics = TaskMetrics(spark)

partitionsize = 200000
thread_workers = 32


In [6]:
import pyspark.sql.functions as fn
from tqdm import tqdm
import json

fileslice = slice(None)
with open("metadata/2018.json") as fin:
    samplefiles = json.load(fin)

filelist = {}
i = 0
for dataset, info in samplefiles.items():
    i = i + 1
    print(i)
    if i > 10: continue
    files = []
    for file in info['files'][fileslice]:
        files.append(file)
        if 'MET____0_' in dataset or 'MET____1_' in dataset: 
            filelist[dataset] = {'files': files, 'treename': 'Events'}
print(filelist['MET____2_'])


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189


KeyError: 'MET____2_'

In [3]:
#get the dark Higgs analysis worker
from coffea.util import load

processor_instance = load('processors/darkhiggs2018.coffea')


In [4]:
import time
from coffea.processor import run_spark_job
from coffea.processor.spark.spark_executor import spark_executor

tic = time.time()
final_accumulator = run_spark_job(filelist, processor_instance, spark_executor, 
                                  spark=spark, partitionsize=partitionsize, thread_workers=thread_workers,
                                  executor_args={'file_type': 'root', 'cache': False})
dt = time.time() - tic


/usr/local/lib64/python3.6/site-packages/pyarrow/pandas_compat.py:751: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels
/usr/local/lib64/python3.6/site-packages/pyarrow/pandas_compat.py:751: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels
/usr/local/lib64/python3.6/site-packages/pyarrow/pandas_compat.py:751: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels
/usr/local/lib64/python3.6/site-packages/pyarrow/pandas_compat.py:751: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels
/usr/local/lib64/python3.6/site-packages/pyarrow/pandas_compat.py:751: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [ ]:
nevt = sum(spark_executor.counts.values())
print('processed:',nevt,'events')
print('total time: ',dt/60)
print('μs/evt', dt/nevt*1e6)
print('Mevt/s', nevt/dt/1e6)


In [ ]:
from coffea import hist
from coffea.util import save
import numpy as np

nbins = sum(sum(arr.size for arr in h._sumw.values()) for h in final_accumulator.values() if isinstance(h, hist.Hist))
nfilled = sum(sum(np.sum(arr>0) for arr in h._sumw.values()) for h in final_accumulator.values() if isinstance(h, hist.Hist))
print("Processed %.1fM events" % (nevt/1e6, ))
print("Filled %.1fM bins" % (nbins/1e6, ))
print("Nonzero bins: %.1f%%" % (100*nfilled/nbins, ))

# Pickle is not very fast or memory efficient, will be replaced by something better soon
save(final_accumulator,'pods/pods2018.coffea')

#dt = time.time() - tstart
#print("%.2f us*cpu/event overall" % (1e6*dt*nworkers/final_accumulators['nentries'], ))


In [ ]:
spark.stop()
